In [2]:
import pandas as pd
import numpy as np
from math import pi, cos, sin, sqrt
from sklearn.ensemble import RandomForestClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import SVC
from sklearn.linear_model import LogisticRegression
from sklearn.feature_selection import SelectKBest, f_classif
from sklearn.decomposition import PCA
import matplotlib.pyplot as plt
# Load pylab
%pylab inline

Populating the interactive namespace from numpy and matplotlib


C:\Users\jordan\Anaconda2\lib\site-packages\IPython\core\magics\pylab.py:161: UserWarning: pylab import has clobbered these variables: ['cos', 'sqrt', 'pi', 'sin']
`%matplotlib` prevents importing * from pylab and numpy
  "\n`%matplotlib` prevents importing * from pylab and numpy"


In [3]:
#Our loss evaluation
def log_loss(y_true, y_pred):
    """
    Parameters
    ----------
    y_true : array, shape = [n_samples]
    y_pred : array, shape = [n_samples]

    Returns
    -------
    loss : float
    """
    prediction = sqrt(np.average((log(y_true.astype(int) + 1)  - log(y_pred.astype(int) + 1)) ** 2))
    return prediction

In [4]:
'''# Load the bike data
X = np.loadtxt('data/train.csv',  delimiter=',', 
               skiprows=1, usecols=range(0,13),converters = {1: datestr2num})
y = np.loadtxt('data/train.csv',  delimiter=',', 
               skiprows=1, usecols=[14])
X_real_test = np.loadtxt('data/test.csv',  delimiter=',', 
               skiprows=1, usecols=range(0,13),converters = {1: datestr2num})'''

"# Load the bike data\nX = np.loadtxt('data/train.csv',  delimiter=',', \n               skiprows=1, usecols=range(0,13),converters = {1: datestr2num})\ny = np.loadtxt('data/train.csv',  delimiter=',', \n               skiprows=1, usecols=[14])\nX_real_test = np.loadtxt('data/test.csv',  delimiter=',', \n               skiprows=1, usecols=range(0,13),converters = {1: datestr2num})"

In [5]:
test_df = pd.read_csv("data/test.csv",header=0)
train_df = pd.read_csv("data/train.csv",header=0)
del train_df['dteday'],train_df['holiday'], train_df['atemp'], train_df['workingday']
'''from sklearn.cross_validation import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=42)
print y_test'''

'from sklearn.cross_validation import train_test_split\nX_train, X_test, y_train, y_test = train_test_split(X, y, random_state=42)\nprint y_test'

In [6]:
train_df.corr()

,instant,season,yr,mnth,hr,weekday,weathersit,temp,hum,windspeed,cnt
instant,1.000000,0.479463,0.866974,0.493368,-0.005638,-0.016307,-0.005118,0.180773,0.032624,-0.086933,0.310185
season,0.479463,1.000000,-0.004797,0.971524,-0.006546,-0.009691,0.008879,0.258689,0.190610,-0.147121,0.163439
yr,0.866974,-0.004797,1.000000,-0.004932,-0.004234,-0.018162,-0.012548,0.061226,-0.078606,-0.015221,0.260403
mnth,0.493368,0.971524,-0.004932,1.000000,-0.006818,-0.001012,0.012144,0.257589,0.204537,-0.150192,0.166862
hr,-0.005638,-0.006546,-0.004234,-0.006818,1.000000,-0.001660,-0.022740,0.145430,-0.278011,0.146631,0.400601
weekday,-0.016307,-0.009691,-0.018162,-0.001012,-0.001660,1.000000,-0.016439,-0.026143,-0.059722,-0.006178,0.027690
weathersit,-0.005118,0.008879,-0.012548,0.012144,-0.022740,-0.016439,1.000000,-0.055035,0.406244,0.007261,-0.128655
temp,0.180773,0.258689,0.061226,0.257589,0.145430,-0.026143,-0.055035,1.000000,-0.064949,-0.017852,0.394454
hum,0.032624,0.190610,-0.078606,0.204537,-0.278011,-0.059722,0.406244,-0.064949,1.000000,-0.318607,-0.317371
windspeed,-0.086933,-0.147121,-0.015221,-0.150192,0.146631,-0.006178,0.007261,-0.017852,-0.318607,1.000000,0.101369


In [7]:
def binarization(data,column) :
    '''Creation of dummy variables'''
    for unique in data[column].unique() :
        data[column+'='+str(unique)] = data[column].apply(lambda x : 1 if x == unique else 0)

In [8]:
#Binarization of weekday, season, weathersit
for column in ['season','weathersit'] :
    binarization(train_df,column)
    binarization(test_df,column)

In [9]:
def categorization(data,column) :
    '''Creation of category variables'''
    data[column+'_cat'] = data[column].astype('category')

In [12]:
for column in [el for el in train_df.columns if '=' in el] + ['yr'] :
    categorization(train_df,column)
    categorization(test_df,column)

In [25]:
train_df.describe()

,instant,season,yr,mnth,hr,weekday,weathersit,temp,hum,windspeed,cnt,season=1,season=2,season=3,season=4,weathersit=1,weathersit=2,weathersit=3,weathersit=4
count,10886.000000,10886.000000,10886.000000,10886.000000,10886.000000,10886.000000,10886.000000,10886.000000,10886.000000,10886.000000,10886.000000,10886.000000,10886.000000,10886.000000,10886.000000,10886.000000,10886.000000,10886.000000,10886.000000
mean,8537.421183,2.506614,0.501929,6.521495,11.541613,2.998622,1.418427,0.493436,0.618865,0.191036,191.574132,0.246739,0.251056,0.251056,0.251148,0.660665,0.260334,0.078909,0.000092
std,5026.363303,1.116174,0.500019,3.444373,6.915838,2.007770,0.633839,0.190039,0.192450,0.121859,181.144454,0.431133,0.433641,0.433641,0.433694,0.473505,0.438837,0.269609,0.009584
min,1.000000,1.000000,0.000000,1.000000,0.000000,0.000000,1.000000,0.020000,0.000000,0.000000,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,4282.250000,2.000000,0.000000,4.000000,6.000000,1.000000,1.000000,0.340000,0.470000,0.104500,42.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
50%,8666.500000,3.000000,1.000000,7.000000,12.000000,3.000000,1.000000,0.500000,0.620000,0.194000,145.000000,0.000000,0.000000,0.000000,0.000000,1.000000,0.000000,0.000000,0.000000
75%,13016.750000,4.000000,1.000000,10.000000,18.000000,5.000000,2.000000,0.640000,0.770000,0.253700,284.000000,0.000000,1.000000,1.000000,1.000000,1.000000,1.000000,0.000000,0.000000
max,17093.000000,4.000000,1.000000,12.000000,23.000000,6.000000,4.000000,1.000000,1.000000,0.850700,977.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000


In [49]:
#Standardization of numerical features
def standardization(data,column) :
    '''Standardization of our numerical features'''
    mean_col = data[column].mean()
    std = data[column].std()
    data[column+'_std'] = data[column].apply(lambda x: (x - mean_col)/std)

In [61]:
for column in ['instant','yr','windspeed','hum','temp']+[el for el in train_df.columns if 'cat' in el] :
    standardization(train_df,column)
    standardization(test_df,column)

In [62]:
#Cyclic featuring: Useful for dates: for example we want 'Monday' to be close to 'Sunday' => cosine
def cyclic_featuring(data) :
    '''Cyclic features'''
    d_day = {'0' : 0, '1': 2*pi/7, '2' : 4*pi/7, '3' : 6*pi/7, '4' : 8*pi/7, '5' : 10*pi/7,
     '6' : 12*pi/7}
    
    d_hour = {}
    for h in range(25) :
        d_hour[h] = 2*pi*h/24
    
    d_month = {}
    for m in range(1,13) :
        d_month[m] = 2*pi*m/12

    data['cosDay'] = data['weekday'].apply(lambda x: cos(d_day[str(x)]))
    data['sinDay'] = data['weekday'].apply(lambda x: sin(d_day[str(x)]))
    data['cosHour'] = data['hr'].apply(lambda x: cos(d_hour[x]))
    data['sinHour'] = data['hr'].apply(lambda x: sin(d_hour[x]))
    data['cosMonth'] = data['mnth'].apply(lambda x: cos(d_month[x]))
    data['sinMonth'] = data['mnth'].apply(lambda x: sin(d_month[x]))

In [63]:
cyclic_featuring(train_df)
cyclic_featuring(test_df)

In [64]:
features =[el for el in train_df.columns if 'cat' in el] + [col for col in train_df.columns if 'std' in col] + [col for col in train_df.columns if 'sin' in col] + [col for col in train_df if 'cos' in col]

#We can select the feature either by statistical filtering....
select = SelectKBest(f_classif, k='all')
X_new = pd.DataFrame(select.fit_transform(train_df.loc[:,features], train_df.cnt))

features_selected=[]
for element in features :
    if select.get_support()[features.index(element)] :
        features_selected.append(element)

In [65]:
print features_selected
train_df.describe()

['season=1', 'season=2', 'season=3', 'season=4', 'weathersit=1', 'weathersit=2', 'weathersit=3', 'weathersit=4', 'season=1_std', 'season=2_std', 'season=3_std', 'season=4_std', 'weathersit=1_std', 'weathersit=2_std', 'weathersit=3_std', 'weathersit=4_std', 'instant_std', 'yr_std', 'windspeed_std', 'hum_std', 'temp_std', 'season=1_std', 'season=2_std', 'season=3_std', 'season=4_std', 'weathersit=1_std', 'weathersit=2_std', 'weathersit=3_std', 'weathersit=4_std', 'sinDay', 'sinHour', 'sinMonth', 'cosDay', 'cosHour', 'cosMonth']


,instant,season,yr,mnth,hr,weekday,weathersit,temp,hum,windspeed,...,hum_std,temp_std,season=1_std,season=2_std,season=3_std,season=4_std,weathersit=1_std,weathersit=2_std,weathersit=3_std,weathersit=4_std
count,10886.000000,10886.000000,10886.000000,10886.000000,10886.000000,10886.000000,10886.000000,10886.000000,10886.000000,10886.000000,...,1.088600e+04,1.088600e+04,1.088600e+04,1.088600e+04,1.088600e+04,1.088600e+04,1.088600e+04,1.088600e+04,1.088600e+04,1.088600e+04
mean,8537.421183,2.506614,0.501929,6.521495,11.541613,2.998622,1.418427,0.493436,0.618865,0.191036,...,4.089840e-14,-4.802770e-15,-8.096530e-15,2.276147e-14,2.161280e-14,-5.046079e-15,2.168943e-15,-9.962227e-16,-2.471638e-15,-1.303340e-15
std,5026.363303,1.116174,0.500019,3.444373,6.915838,2.007770,0.633839,0.190039,0.192450,0.121859,...,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00
min,1.000000,1.000000,0.000000,1.000000,0.000000,0.000000,1.000000,0.020000,0.000000,0.000000,...,-3.215711e+00,-2.491258e+00,-5.723031e-01,-5.789501e-01,-5.789501e-01,-5.790915e-01,-1.395264e+00,-5.932368e-01,-2.926787e-01,-9.584420e-03
25%,4282.250000,2.000000,0.000000,4.000000,6.000000,1.000000,1.000000,0.340000,0.470000,0.104500,...,-7.735222e-01,-8.073910e-01,-5.723031e-01,-5.789501e-01,-5.789501e-01,-5.790915e-01,-1.395264e+00,-5.932368e-01,-2.926787e-01,-9.584420e-03
50%,8666.500000,3.000000,1.000000,7.000000,12.000000,3.000000,1.000000,0.500000,0.620000,0.194000,...,5.899721e-03,3.454240e-02,-5.723031e-01,-5.789501e-01,-5.789501e-01,-5.790915e-01,7.166443e-01,-5.932368e-01,-2.926787e-01,-9.584420e-03
75%,13016.750000,4.000000,1.000000,10.000000,18.000000,5.000000,2.000000,0.640000,0.770000,0.253700,...,7.853216e-01,7.712342e-01,-5.723031e-01,1.727106e+00,1.727106e+00,1.726684e+00,7.166443e-01,1.685513e+00,-2.926787e-01,-9.584420e-03
max,17093.000000,4.000000,1.000000,12.000000,23.000000,6.000000,4.000000,1.000000,1.000000,0.850700,...,1.980435e+00,2.665584e+00,1.747165e+00,1.727106e+00,1.727106e+00,1.726684e+00,7.166443e-01,1.685513e+00,3.416402e+00,1.043264e+02


In [66]:
'''The methods'''

LNR = LogisticRegression(C = 1e4, n_jobs=-1)
LR = LogisticRegression(C = 50, n_jobs=-1)
kNN = KNeighborsClassifier(n_neighbors=5500, n_jobs=-1)
RF = RandomForestClassifier(n_jobs=-1, n_estimators=450, min_samples_split=400)
svm = SVC(C= 0.01, probability=True)

In [ ]:
def cross_validate(design_matrix, cnt, method, cv_folds):
    pred = np.zeros(cnt.shape)
    matrix_log_loss = np.zeros(1)
    for tr, te in cv_folds:
        # Restrict data to train/test folds
        Xtr = design_matrix[tr, :]
        ytr = labels[tr]
        Xte = design_matrix[te, :]
        #print Xtr.shape, ytr.shape, Xte.shape
        # Fit classifier
        method.fit(Xtr, ytr)

        # Predict probabilities (of belonging to +1 class) on test data
        yte_pred = method.predict(Xte) # two-dimensional array
        # Identify the index, in yte_pred, of the positive class (y=1)
        print log_loss(ytr,yte_pred)
        matrix_log_loss[te] = log_loss(ytr,yte_pred)
        pred[te] = yte_pred
    return pred,matrix_log_loss

In [ ]:
'''Fit method'''
#Chose your estimator (LR, LNR, kNN, RF,svm)
estimator = LR
cross_validate (train_df.loc[:,features_selected],train_df.cnt,estimator,10)

In [ ]:
estimator.score(train_df.loc[:,features_selected], train_df.cnt)

In [ ]:
log_loss(train_df.cnt, est)